<a href="https://colab.research.google.com/github/Carinaaa/AI-Models-Problems/blob/main/feedback_summarizer_analyzer/Feedback_summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import userdata
from openai import OpenAI
import os
import csv

In [ ]:
openai_key = userdata.get('OPENAI_API_KEY') # get the secret by the name
os.environ['OPENAI_API_KEY'] = openai_key # set it as an env var
openai = OpenAI() # create model using the env var
openai_model = "gpt-4o-mini" # "gpt-5.1"
anthropic_model = "claude-sonnet-4-5"
google_model = "gemini-2.5-pro"

In [ ]:
questions = ['1. Can you share a general impression?', '2. What did you like the most from this workshop?', '3. How was the interraction with the trainer?',
             '4. What do you think about the teaching method?', '5. What expectations did you have?' ,'6. What would you improve?']

with open("workshop_feedback_data.csv", "w", newline="", encoding="utf-8") as file:
  writer = csv.writer(file)
  writer.writerow(questions)


In [ ]:
impressions = ['content', 'disappointed', "collaborative", "common", "supportive", "sleepy", "tired", "annoyed"]
occupations = ['SW Engineer', 'Car Owner', "teacher", 'HW engineer', "Chassis Designer", "Data analyst", "Student", "Team Manager"]
studies = ['Computer Science', "Mechanical Engineering", "Artificial Intelligence", "Electrotechnical Engineering", "Street Vehicles", "Math", "Computer Science", "Mechanical Engineering"]

with open("workshop_feedback_data.csv", "a", newline="", encoding="utf-8") as file:
  for p in range(len(impressions)):
    messages = [
        {"role": "system", "content": f"You are a {impressions[p]} participant, which is a {occupations[p]} and studied {studies[p]} to a workshop dedicated to ADAS Systems where a prototype has been presented from HW and SW side."},
        {"role": "user", "content": f"Can you please make some responses, on separated lines, to the following questions: {questions}"}
    ]
    response = openai.chat.completions.create(model=openai_model, messages=messages)
    # print(response.choices[0].message.content)
    answers = response.choices[0].message.content.split('\n')
    answers = [a for a in answers if a != '']
    writer = csv.writer(file)
    writer.writerow(answers)
    print("Writing responseses...")

print("Done!")


Writing responseses...
Writing responseses...
Writing responseses...
Writing responseses...
Writing responseses...
Writing responseses...
Writing responseses...
Writing responseses...
Done!


In [ ]:
import pandas as pd

# Read the CSV file into a pandas DataFrame
df = pd.read_csv("workshop_feedback_data.csv")

# Convert the DataFrame to a string representation for the AI model
# Using df.to_csv(index=False) provides a clean CSV string
feedback_content = df.to_csv(index=False)

feedback_points = ['overall satisfaction rating', 'general topics highlighted', 'how many people noticed the same things', 'strengths highlighted', 'highlighted mistakes', 'familiarity with the workshop theme', 'emotion type', 'biases', 'summary']

messages = [
    {"role": "system", "content": "You are a feedback analyst."},
    {"role": "user", "content": f"Can you please make a report containg: {feedback_points}, from the following feedback data in CSV format:\n{feedback_content}"}
]


In [ ]:
# GPT-5 report
response = openai.chat.completions.create(model=openai_model, messages=messages)
with open("GPT-summ.txt", "w") as file:
  file.write(response.choices[0].message.content)


In [ ]:
# Claude report
import anthropic

anthropic_key = userdata.get('CLAUDE_API_KEY')

client = anthropic.Anthropic(api_key=anthropic_key)

message = client.messages.create(
    model=anthropic_model,
    max_tokens=1000,
    messages=[messages[1]]
)

with open("Claude-summ.txt", "w") as file:
  file.write(message.content[0].text)



In [ ]:
# Deepseek report
deepseek_key = userdata.get('DEEPSEEK_KEY')
deepseek_via_openai_client = OpenAI(
    api_key=deepseek_key,
    base_url="https://api.deepseek.com"
)

response = deepseek_via_openai_client.chat.completions.create(
    model="deepseek-chat",
    messages=messages,
)

with open("DeepSeek-summ.txt", "w") as file:
  file.write(response.choices[0].message.content)


In [ ]:
with open("GPT-summ.txt", "r") as file:
  gpt_summ = file.read()

with open("Claude-summ.txt", "r") as file:
  claude_summ = file.read()

with open("DeepSeek-summ.txt", "r") as file:
  deepseek_summ = file.read()

messages_report_check = [
    {"role": "system", "content": "You are a feedback analyst."},
    {"role": "user", "content": f"Can you please make a comparison checking for differences and common observations, between those 3 feedback summarizations:\
    first analysis: {gpt_summ}, second analysis {claude_summ}, third analysis {deepseek_summ} "}
]

In [ ]:
# Google analysis on the 3 reports, find differences, and common observations
google_key = userdata.get('GEMINI_API_KEY')
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"

gemini = OpenAI(base_url=GEMINI_BASE_URL, api_key=google_key)

response = gemini.chat.completions.create(model=google_model, messages=messages_report_check)

with open("Google-reports-check.txt", "w") as file:
  file.write(response.choices[0].message.content)